<a href="https://colab.research.google.com/github/abhi-gm/Machine-Learning-Workshop/blob/Classical-Machine-Learning/2_Decision_tress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. Decision tress**

##**1.Abstract**

**Churn rate**, is the rate at which customers stop doing business with an entity. It is most commonly expressed as the percentage of service subscribers who discontinue their subscriptions within a given time. The notebook aims to do prediction whether a customer will churn or not.

## **2.Importing necessary libraries**

In [1]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from pandas import DataFrame

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


## **3. Import the data from Github**

In [3]:
url =  'https://raw.githubusercontent.com/abhi-gm/Machine-Learning-Workshop/main/Datasets/Churn.csv'
data = pd.read_csv(url, error_bad_lines=False)

In [4]:
data.head()

,CustomerID,Gender,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
data.shape

(6499, 21)

## **4.Preprocessing the data**

### **4.1.Droping unique columns**

In [ ]:
data = data.drop('CustomerID',axis =1)

### **4.2.Encoding the categorical columns**

In [ ]:
data.dtypes

In [ ]:
categorical_columns = data.select_dtypes(include=['object']).columns

continous_columns = data.select_dtypes(exclude=['object']).columns


In [ ]:
#the categorical columns
categorical_columns

In [ ]:
#making categorical columns into categorical data type
data[categorical_columns] = data[categorical_columns].astype('category')

In [ ]:
data.dtypes

In [ ]:
#dictionary to store mapping
col_cat_dict = {}
#numerical encoding the catecogrical columns using function cat codes
for col in categorical_columns:
  col_cat_dict[col] = dict(enumerate(data[col].cat.categories))
  data[col] = data[col].cat.codes
  data[col] = data[col].astype('category')

In [ ]:
col_cat_dict

In [ ]:
data.dtypes

In [ ]:
data.head()

## **4.3 Handling missing values**

Filling the NaN values for the categorical column with mode
 
Filling the NAN vaules for the continous columns with mean

In [ ]:
for col in categorical_columns:
  data[col] = data[col].fillna(data[col].mode())

for col in continous_columns:
  data[col] = data[col].fillna(data[col].mean())

In [ ]:
#Checking wether there is any NAN values
data.isnull().sum() 

## **5.Training to get the variable importance**

In [ ]:
feature = data.columns.tolist()
feature.remove('Churn')

X = data[ feature]

y = data['Churn']

In [ ]:
dtree=DecisionTreeClassifier()
dtree.fit(X,y)

## **6.1.Feature Importance**

In [ ]:
importance = dtree.feature_importances_
from matplotlib import pyplot

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))


pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()
var_imp = importance.tolist()

In [ ]:
feature = data.columns.tolist()
feature.remove('Churn')

df_varimp =  DataFrame (feature,columns=['Features'])
df_varimp["var_imp"] =var_imp
df_varimp_sorted=df_varimp.sort_values(by=['var_imp'],ascending=False)
df_varimp_sorted



## **6.2.Feature Selection**

thres is set 0.05 which configurable

In [ ]:
thres = 0.05
imp_data = df_varimp_sorted[df_varimp_sorted['var_imp']>thres]
feat = imp_data['Features']


In [ ]:
data [feat]

### **7.1.Train ,Validation and Test split**

Data is split into 3 parts

Taining data set = 80.75%

Validation data set = 14.25%

Test data set = 5%

In [ ]:
from sklearn.model_selection import  train_test_split

columns = data[feat].columns

X = data[feat]

y = data['Churn']

X_t, X_test, y_t, y_test = train_test_split(X, y, test_size=0.05, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_t, y_t, test_size=0.15, random_state=1)

### **7.2.Processing data for decision tree**

In [ ]:
feature = data.columns
y = data['Churn']
data = data.drop(('Churn'),axis = 1)
data = data.fillna(0)

## **8. Tunning the Hyperparmeter**

Using grid search for Hyperparmeter tunning, even randomized grid search can be employed for doing grid search which will faster than normal grid search.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
depths = np.arange(2, 5)
num_leafs = [ 20,30,40,50]
nodes =np.arange(24, 30)
criteria = ['entropy']
split = ['best', 'random']

In [ ]:
param_grid = [{'max_depth':depths,
              'min_samples_leaf':num_leafs,'max_leaf_nodes':nodes,'criterion':criteria}]

In [ ]:
grid = GridSearchCV(dtree, param_grid, cv=10, scoring='accuracy',verbose=1)

In [ ]:
s = grid.fit(X_val, y_val)

In [ ]:
# examine the best model

# Single best score achieved across all params (min_samples_split)
print(grid.best_score_)

# Dictionary containing the parameters (min_samples_split) used to generate that score
print(grid.best_params_)

# Actual model object fit with those best parameters
# Shows default parameters that we did not specify
print(grid.best_estimator_)

## **9. Training important Features**


After the hyperparameter tunning

**max_depth**: 3

**max_leaf_nodes**: 24 

**min_samples_leaf**: 50

In [ ]:
dtree=DecisionTreeClassifier(max_depth= 3, max_leaf_nodes= 24, min_samples_leaf= 50,criterion='entropy')
model = dtree.fit(X_train,y_train)

In [ ]:
y_predicted = model.predict(X_test)
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_predicted))


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predicted)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

plot_confusion_matrix(model, X_test, y_test)  
plt.show() 

## **10. Conclusion**

So finally the model predictions is good and we can predict customer churn with 76% accuracy.

## **11.Reference**

https://scikit-learn.org/stable/modules/tree.html

Copyright 2020 Abhishek Gargha Maheshwarappa

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.